In [ ]:
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import pandas as pd
import pandas_datareader as web
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
# Dataset
df_confirmed = pd.read_csv(
    "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

In [ ]:
# Data Preparation
df_confirmed = df_confirmed.loc[df_confirmed['Country/Region']== 'Indonesia']

# Removing Useless Columns
df_confirmed.drop('Province/State', axis=1, inplace=True)
df_confirmed.drop('Lat', axis=1, inplace=True)
df_confirmed.drop('Long', axis=1, inplace=True)

df_confirmed.head()

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,...,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21,7/28/21,7/29/21,7/30/21,7/31/21,8/1/21,8/2/21,8/3/21,8/4/21,8/5/21,8/6/21,8/7/21,8/8/21,8/9/21,8/10/21,8/11/21,8/12/21,8/13/21,8/14/21,8/15/21,8/16/21,8/17/21,8/18/21,8/19/21
148,Indonesia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2527203,2567630,2615529,2670046,2726803,2780803,2832755,2877476,2911733,2950058,2983830,3033339,3082410,3127826,3166505,3194733,3239936,3287727,3331206,3372374,3409658,3440396,3462800,3496700,3532567,3568331,3607863,3639616,3666031,3686740,3718821,3749446,3774155,3804943,3833541,3854354,3871738,3892479,3908247,3930300


In [ ]:
# Group all rows of the same country
df_confirmed = df_confirmed.groupby('Country/Region').sum()

In [ ]:
# Remove Zero Values
df_confirmed = df_confirmed.loc[:, (df_confirmed !=0).any(axis=0)]

In [ ]:
# Remove the Unused Country Column
df_confirmed.reset_index(inplace=True)
df_confirmed.drop('Country/Region', axis=1, inplace=True)
df_confirmed.head()

,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,...,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21,7/19/21,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21,7/28/21,7/29/21,7/30/21,7/31/21,8/1/21,8/2/21,8/3/21,8/4/21,8/5/21,8/6/21,8/7/21,8/8/21,8/9/21,8/10/21,8/11/21,8/12/21,8/13/21,8/14/21,8/15/21,8/16/21,8/17/21,8/18/21,8/19/21
0,2,2,2,2,4,4,6,19,27,34,34,69,96,117,134,172,227,311,369,450,514,579,686,790,893,1046,1155,1285,1414,1528,1677,1790,1986,2092,2273,2491,2738,2956,3293,3512,...,2527203,2567630,2615529,2670046,2726803,2780803,2832755,2877476,2911733,2950058,2983830,3033339,3082410,3127826,3166505,3194733,3239936,3287727,3331206,3372374,3409658,3440396,3462800,3496700,3532567,3568331,3607863,3639616,3666031,3686740,3718821,3749446,3774155,3804943,3833541,3854354,3871738,3892479,3908247,3930300


In [ ]:
# Transpose the matrix to create a row for each date entry
df_confirmed = df_confirmed.transpose()

In [ ]:
# Change columns name
df_confirmed.reset_index(inplace=True)
df_confirmed.rename(columns={'index':'Date',0:'confirmed'}, inplace=True)
df_confirmed.head()

,Date,confirmed
0,3/2/20,2
1,3/3/20,2
2,3/4/20,2
3,3/5/20,2
4,3/6/20,4


In [ ]:
# Convert datetime format
df_confirmed['Date'] = pd.to_datetime(df_confirmed['Date'], format = '%m/%d/%y').dt.strftime('%Y-%m-%d')
df_confirmed

,Date,confirmed
0,2020-03-02,2
1,2020-03-03,2
2,2020-03-04,2
3,2020-03-05,2
4,2020-03-06,4
...,...,...
531,2021-08-15,3854354
532,2021-08-16,3871738
533,2021-08-17,3892479
534,2021-08-18,3908247


In [ ]:
mask = (df_confirmed['Date']>'2020-03-02') & (df_confirmed['Date']<'2021-03-04')
df_confirmed = df_confirmed.loc[mask]

In [ ]:
df_confirmed

,Date,confirmed
1,2020-03-03,2
2,2020-03-04,2
3,2020-03-05,2
4,2020-03-06,4
5,2020-03-07,4
...,...,...
362,2021-02-27,1329074
363,2021-02-28,1334634
364,2021-03-01,1341314
365,2021-03-02,1347026


In [ ]:
# Group dataset based on the date
groupDataset = df_confirmed.groupby('Date')
data = []
date = []
cases = []
for name, df_confirmed in groupDataset:
  print(name)
  date.append(name)
  s = sum(df_confirmed['confirmed'].astype(float))
  cases.append(s)
  data.append([s])
data

2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-04
2020-04-05
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-11
2020-04-12
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-18
2020-04-19
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-25
2020-04-26
2020-04-27
2020-04-28
2020-04-29
2020-04-30
2020-05-01
2020-05-02
2020-05-03
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-08
2020-05-09
2020-05-10
2020-05-11
2020-05-12
2020-05-13
2020-05-14
2020-05-15
2020-05-16
2020-05-17
2020-05-18
2020-05-19
2020-05-20
2020-05-21
2020-05-22
2020-05-23
2020-05-24
2020-05-25
2020-05-26
2020-05-27
2020-05-28
2020-05-29
2020-05-30
2020-05-31
2020-06-01

[[2.0],
 [2.0],
 [2.0],
 [4.0],
 [4.0],
 [6.0],
 [19.0],
 [27.0],
 [34.0],
 [34.0],
 [69.0],
 [96.0],
 [117.0],
 [134.0],
 [172.0],
 [227.0],
 [311.0],
 [369.0],
 [450.0],
 [514.0],
 [579.0],
 [686.0],
 [790.0],
 [893.0],
 [1046.0],
 [1155.0],
 [1285.0],
 [1414.0],
 [1528.0],
 [1677.0],
 [1790.0],
 [1986.0],
 [2092.0],
 [2273.0],
 [2491.0],
 [2738.0],
 [2956.0],
 [3293.0],
 [3512.0],
 [3842.0],
 [4241.0],
 [4557.0],
 [4839.0],
 [5136.0],
 [5516.0],
 [5923.0],
 [6248.0],
 [6575.0],
 [6760.0],
 [7135.0],
 [7418.0],
 [7775.0],
 [8211.0],
 [8607.0],
 [8882.0],
 [9096.0],
 [9511.0],
 [9771.0],
 [10118.0],
 [10551.0],
 [10843.0],
 [11192.0],
 [11587.0],
 [12071.0],
 [12438.0],
 [12776.0],
 [13112.0],
 [13645.0],
 [14032.0],
 [14265.0],
 [14749.0],
 [15438.0],
 [16006.0],
 [16496.0],
 [17025.0],
 [17514.0],
 [18010.0],
 [18496.0],
 [19189.0],
 [20162.0],
 [20796.0],
 [21745.0],
 [22271.0],
 [22750.0],
 [23165.0],
 [23851.0],
 [24538.0],
 [25216.0],
 [25773.0],
 [26473.0],
 [26940.0],
 [27549.

In [ ]:
len(data)

366

In [ ]:
dataset = np.array(data)
dataset.shape

(366, 1)

In [ ]:
dataset.shape

(366, 1)

In [ ]:
for i in range(len(cases)-1, 1, -1):
  cases[i] = cases[i] - cases[i-1]
  dataset[i] [0] = cases[i]

In [ ]:
# graph shown
trace = go.Scatter(
    x = date,
    y = cases,
    mode = 'lines',
    name = 'Data'
)
layout = go.Layout(
    title = "Covid-19 in Indonesia",
    xaxis = { 'title' : "Date"},
    yaxis = { 'title' : "Confirmed Cases"}
)
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [ ]:
# Scaling dataset
scaler = MinMaxScaler(feature_range=(200,1000))
dataset = scaler.fit_transform(np.array(dataset).reshape(-1,1))
dataset.shape

(366, 1)

In [ ]:
# Split dataset for training and testing
# using 70% dataset for testing
split_percent = 0.70
split = int(split_percent*len(dataset))
split

256

In [ ]:
dataset_train = dataset[:split]
dataset_test = dataset[split:]

date_train = date[:split]
date_test = date[split:]

print(len(dataset_train))
print(len(dataset_test))

256
110


In [ ]:
look_back = 3
train_generator = TimeseriesGenerator(dataset_train,dataset_train, length=look_back, batch_size=1)
test_generator = TimeseriesGenerator(dataset_test, dataset_test, length=look_back, batch_size=1)
train_generator

In [ ]:
test_generator.__len__()

107

In [ ]:
train_generator.__len__()

253

In [ ]:
train_generator.__getitem__(251)

(array([[[357.21173715],
         [408.23804932],
         [407.74211324]]]), array([[429.94902879]]))

In [ ]:
test_generator.__getitem__(107)

(array([], dtype=float64), array([], dtype=float64))

In [ ]:
model = Sequential()
model.add(LSTM(10, return_sequences=True, activation='relu', input_shape=(look_back,1)))
model.add(LSTM(7, return_sequences=True, activation='relu'))
model.add(LSTM(3, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 10)             480       
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 7)              504       
_________________________________________________________________
lstm_2 (LSTM)                (None, 3)                 132       
_________________________________________________________________
dense (Dense)                (None, 1)                 4         
Total params: 1,120
Trainable params: 1,120
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epoch = 100
model.fit_generator(train_generator, epochs=num_epoch, steps_per_epoch=80, verbose=1)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



80/80 [==============================] - 5s 9ms/step - loss: 99715.8984
Epoch 2/100
80/80 [==============================] - 1s 7ms/step - loss: 58647.9883
Epoch 3/100
80/80 [==============================] - 1s 9ms/step - loss: 574.2632
Epoch 4/100
80/80 [==============================] - 1s 9ms/step - loss: 388.2087
Epoch 5/100
80/80 [==============================] - 1s 7ms/step - loss: 500.5858
Epoch 6/100
80/80 [==============================] - 1s 9ms/step - loss: 439.2206
Epoch 7/100
80/80 [==============================] - 1s 9ms/step - loss: 1523.5872
Epoch 8/100
80/80 [==============================] - 1s 8ms/step - loss: 488.8401
Epoch 9/100
80/80 [==============================] - 1s 8ms/step - loss: 422.4784
Epoch 10/100
80/80 [==============================] - 1s 8ms/step - loss: 571.4186
Epoch 11/100
80/80 [==============================] - 1s 8ms/step - loss: 485.4657
Epoch 12/100
80/80 [==============================] - 1s 8ms/step - loss: 530.9870
Epoch 13/100
80/80 [

In [ ]:
prediction = model.predict_generator(test_generator)
prediction

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2035: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



array([[452.8626 ],
       [409.6823 ],
       [411.3073 ],
       [435.33994],
       [444.0297 ],
       [459.69937],
       [460.00598],
       [452.9181 ],
       [435.60437],
       [489.76013],
       [455.9232 ],
       [496.3501 ],
       [484.28235],
       [515.40814],
       [505.17838],
       [496.379  ],
       [470.8591 ],
       [591.51746],
       [550.4498 ],
       [584.4329 ],
       [516.2822 ],
       [522.22754],
       [515.7431 ],
       [506.80396],
       [502.9915 ],
       [525.05115],
       [529.7138 ],
       [537.8802 ],
       [534.2261 ],
       [518.25793],
       [526.4975 ],
       [556.8231 ],
       [571.6276 ],
       [589.5231 ],
       [579.77704],
       [593.6499 ],
       [567.42395],
       [577.72266],
       [565.80273],
       [592.51715],
       [583.40924],
       [573.4919 ],
       [552.5042 ],
       [621.9273 ],
       [572.76025],
       [623.0093 ],
       [627.279  ],
       [627.44806],
       [607.8374 ],
       [575.3486 ],


In [ ]:
prediction = scaler.inverse_transform(prediction)
prediction

array([[ 4588.824 ],
       [ 3805.2097],
       [ 3834.6995],
       [ 4270.8315],
       [ 4428.529 ],
       [ 4712.8945],
       [ 4718.4585],
       [ 4589.831 ],
       [ 4275.6304],
       [ 5258.422 ],
       [ 4644.366 ],
       [ 5378.013 ],
       [ 5159.014 ],
       [ 5723.869 ],
       [ 5538.2246],
       [ 5378.538 ],
       [ 4915.4155],
       [ 7105.063 ],
       [ 6359.788 ],
       [ 6976.4966],
       [ 5739.732 ],
       [ 5847.624 ],
       [ 5729.9478],
       [ 5567.7246],
       [ 5498.538 ],
       [ 5898.8657],
       [ 5983.4814],
       [ 6131.6807],
       [ 6065.3677],
       [ 5775.586 ],
       [ 5925.1133],
       [ 6475.4478],
       [ 6744.1123],
       [ 7068.87  ],
       [ 6892.004 ],
       [ 7143.7617],
       [ 6667.826 ],
       [ 6854.7217],
       [ 6638.4053],
       [ 7123.205 ],
       [ 6957.9194],
       [ 6777.944 ],
       [ 6397.0703],
       [ 7656.926 ],
       [ 6764.6665],
       [ 7676.561 ],
       [ 7754.0454],
       [ 7757

In [ ]:
dataset = scaler.inverse_transform(dataset)
dataset

array([[2.0000e+00],
       [2.0000e+00],
       [0.0000e+00],
       [2.0000e+00],
       [0.0000e+00],
       [2.0000e+00],
       [1.3000e+01],
       [8.0000e+00],
       [7.0000e+00],
       [0.0000e+00],
       [3.5000e+01],
       [2.7000e+01],
       [2.1000e+01],
       [1.7000e+01],
       [3.8000e+01],
       [5.5000e+01],
       [8.4000e+01],
       [5.8000e+01],
       [8.1000e+01],
       [6.4000e+01],
       [6.5000e+01],
       [1.0700e+02],
       [1.0400e+02],
       [1.0300e+02],
       [1.5300e+02],
       [1.0900e+02],
       [1.3000e+02],
       [1.2900e+02],
       [1.1400e+02],
       [1.4900e+02],
       [1.1300e+02],
       [1.9600e+02],
       [1.0600e+02],
       [1.8100e+02],
       [2.1800e+02],
       [2.4700e+02],
       [2.1800e+02],
       [3.3700e+02],
       [2.1900e+02],
       [3.3000e+02],
       [3.9900e+02],
       [3.1600e+02],
       [2.8200e+02],
       [2.9700e+02],
       [3.8000e+02],
       [4.0700e+02],
       [3.2500e+02],
       [3.270

In [ ]:
dataset_train = scaler.inverse_transform(dataset_train)
dataset_train

array([[2.000e+00],
       [2.000e+00],
       [0.000e+00],
       [2.000e+00],
       [0.000e+00],
       [2.000e+00],
       [1.300e+01],
       [8.000e+00],
       [7.000e+00],
       [0.000e+00],
       [3.500e+01],
       [2.700e+01],
       [2.100e+01],
       [1.700e+01],
       [3.800e+01],
       [5.500e+01],
       [8.400e+01],
       [5.800e+01],
       [8.100e+01],
       [6.400e+01],
       [6.500e+01],
       [1.070e+02],
       [1.040e+02],
       [1.030e+02],
       [1.530e+02],
       [1.090e+02],
       [1.300e+02],
       [1.290e+02],
       [1.140e+02],
       [1.490e+02],
       [1.130e+02],
       [1.960e+02],
       [1.060e+02],
       [1.810e+02],
       [2.180e+02],
       [2.470e+02],
       [2.180e+02],
       [3.370e+02],
       [2.190e+02],
       [3.300e+02],
       [3.990e+02],
       [3.160e+02],
       [2.820e+02],
       [2.970e+02],
       [3.800e+02],
       [4.070e+02],
       [3.250e+02],
       [3.270e+02],
       [1.850e+02],
       [3.750e+02],


In [ ]:
dataset_test = scaler.inverse_transform(dataset_test)
dataset_test

array([[ 5272.],
       [ 4106.],
       [ 3535.],
       [ 3807.],
       [ 4265.],
       [ 4798.],
       [ 4792.],
       [ 4998.],
       [ 4360.],
       [ 4442.],
       [ 4192.],
       [ 5534.],
       [ 4917.],
       [ 5828.],
       [ 5418.],
       [ 6267.],
       [ 4617.],
       [ 5092.],
       [ 5533.],
       [ 8369.],
       [ 5803.],
       [ 6027.],
       [ 6089.],
       [ 5754.],
       [ 5292.],
       [ 6058.],
       [ 6033.],
       [ 6310.],
       [ 6388.],
       [ 6189.],
       [ 5489.],
       [ 6120.],
       [ 6725.],
       [ 7354.],
       [ 6689.],
       [ 7751.],
       [ 6982.],
       [ 6848.],
       [ 6347.],
       [ 7514.],
       [ 7199.],
       [ 7259.],
       [ 6740.],
       [ 6528.],
       [ 5854.],
       [ 7903.],
       [ 8002.],
       [ 8074.],
       [ 8072.],
       [ 7203.],
       [ 6877.],
       [ 6753.],
       [ 7445.],
       [ 8854.],
       [ 9321.],
       [10617.],
       [10046.],
       [ 9640.],
       [ 8692.

In [ ]:
date_test

['2020-11-14',
 '2020-11-15',
 '2020-11-16',
 '2020-11-17',
 '2020-11-18',
 '2020-11-19',
 '2020-11-20',
 '2020-11-21',
 '2020-11-22',
 '2020-11-23',
 '2020-11-24',
 '2020-11-25',
 '2020-11-26',
 '2020-11-27',
 '2020-11-28',
 '2020-11-29',
 '2020-11-30',
 '2020-12-01',
 '2020-12-02',
 '2020-12-03',
 '2020-12-04',
 '2020-12-05',
 '2020-12-06',
 '2020-12-07',
 '2020-12-08',
 '2020-12-09',
 '2020-12-10',
 '2020-12-11',
 '2020-12-12',
 '2020-12-13',
 '2020-12-14',
 '2020-12-15',
 '2020-12-16',
 '2020-12-17',
 '2020-12-18',
 '2020-12-19',
 '2020-12-20',
 '2020-12-21',
 '2020-12-22',
 '2020-12-23',
 '2020-12-24',
 '2020-12-25',
 '2020-12-26',
 '2020-12-27',
 '2020-12-28',
 '2020-12-29',
 '2020-12-30',
 '2020-12-31',
 '2021-01-01',
 '2021-01-02',
 '2021-01-03',
 '2021-01-04',
 '2021-01-05',
 '2021-01-06',
 '2021-01-07',
 '2021-01-08',
 '2021-01-09',
 '2021-01-10',
 '2021-01-11',
 '2021-01-12',
 '2021-01-13',
 '2021-01-14',
 '2021-01-15',
 '2021-01-16',
 '2021-01-17',
 '2021-01-18',
 '2021-01-

In [ ]:
dataset_train = dataset_train.reshape((-1))
dataset_test = dataset_test.reshape((-1))
print(dataset_train.shape, dataset_test.shape)
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = dataset_train,
    mode = 'lines',
    name = 'Data'
)

trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)

trace3 = go.Scatter(
    x = date_test,
    y = dataset_test,
    mode = 'lines',
    name = 'Ground Truth'
)

layout = go.Layout(
    title = "Covid-19 in Indonesia",
    xaxis = {'title' : 'Date'},
    yaxis = {'title' : 'Confirmed Cases'}
)

fig = go.Figure(data=[trace1,trace2,trace3], layout=layout)
fig.show()

(256,) (110,)


In [ ]:
dataset = dataset.reshape((-1))
dataset.shape

(366,)

In [ ]:
def predict(num_prediction, model):
  prediction_list = dataset[-look_back:]

  for _ in range(num_prediction):
    x = prediction_list[-look_back:]
    x = x.reshape((1, look_back, 1))
    out = model.predict(x)[0][0]
    prediction_list = np.append(prediction_list, out)
  prediction_list = prediction_list[look_back-1:]

  return prediction_list

def predict_dates(num_prediction):
  last_date = date[-1]
  prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
  return prediction_dates

In [ ]:
num_prediction = 7
forecast = predict(num_prediction, model).astype(int)
forecast_dates = predict_dates(num_prediction)

In [ ]:
forecast

array([6808, 6102, 5961, 6264, 5838, 5871, 5865, 5649])

In [ ]:
forecast_dates = [f.date().strftime("%Y-%m-%d") for f in forecast_dates]
forecast_dates

['2021-03-03',
 '2021-03-04',
 '2021-03-05',
 '2021-03-06',
 '2021-03-07',
 '2021-03-08',
 '2021-03-09',
 '2021-03-10']

In [ ]:
given_trace = go.Scatter(
    x = date,
    y = cases,
    mode = 'lines',
    name = 'Data' 
)
forcast_trace = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
layout = go.Layout(
    title = "Covid 19 Indonesia Prediction Information",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Confirmed Cases"}
)
fig = go.Figure(data=[given_trace, forcast_trace], layout=layout)
fig.show()